# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с бустингом, предоставляемым библиотекой `CatBoost`.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека.

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


In [ ]:
import inspect
import random
from collections import Counter
from dataclasses import dataclass
from itertools import product
from typing import Callable, List, Tuple, Union, Optional

import numpy as np
import numpy.typing as npt
import pandas
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)


# Этой функцией будут помечены все места, которые необходимо дозаполнить
# Это могут быть как целые функции, так и отдельные части внутри них
# Всегда можно воспользоваться интроспекцией и найти места использования этой функции :)
def todo():
    stack = inspect.stack()
    caller_frame = stack[1]
    function_name = caller_frame.function
    line_number = caller_frame.lineno
    raise NotImplementedError(f"TODO at {function_name}, line {line_number}")


SEED = 0xC0FFEE
set_seed(SEED)

In [ ]:
def mode(data):
    counts = Counter(data)
    return counts.most_common(n=1)[0][0]

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bootstrap. Элементы, которые не вошли в новую обучающую выборку, образуют **out-of-bag** выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bootstrap. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [ ]:
# Для начала реализуем сами критерии


def gini(x: npt.ArrayLike) -> float:
    """
    Calculate the Gini impurity of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Gini impurity value.
    """
    labels = np.asarray(x)

    class_counts = np.unique(labels, return_counts=True)[1]
    probabilities = class_counts / len(labels)

    return np.sum(probabilities * (1 - probabilities))


def entropy(x: npt.ArrayLike) -> float:
    """
    Calculate the entropy of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Entropy value.
    """
    labels = np.asarray(x)

    class_counts = np.unique(labels, return_counts=True)[1]
    probabilities = class_counts / len(labels)

    return -np.sum(probabilities * np.log2(probabilities))


def gain(left_y: npt.ArrayLike, right_y: npt.ArrayLike, criterion: Callable[[npt.ArrayLike], float]) -> float:
    """
    Calculate the information gain of a split using a specified criterion.

    Args:
        left_y (ArrayLike): Class labels for the left split.
        right_y (ArrayLike): Class labels for the right split.
        criterion (Callable): Function to calculate impurity (e.g., gini or entropy).

    Returns:
        float: Information gain from the split.
    """
    left = np.asarray(left_y)
    right = np.asarray(right_y)

    parent = np.concatenate((left, right))
    parent_impurity = criterion(parent)

    left_impurity = (len(left) / len(parent)) * criterion(left)
    right_impurity = (len(right) / len(parent)) * criterion(right)

    return parent_impurity - left_impurity - right_impurity

In [ ]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)


@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union["DecisionTreeInternalNode", DecisionTreeLeaf]
    right: Union["DecisionTreeInternalNode", DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [ ]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto") -> None:
        self._X = X
        self._y = y
        self._criterion = self._select_crit(criterion)
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = self._validate_max_feat(max_features)
        self._out_of_bag_X = None
        self._out_of_bag_y = None

        self._boot_X, self._boot_y = self._bootstrap_init()

        self._root = self._build_node(self._boot_X, self._boot_y, 0)


    def _bootstrap_init(self) -> tuple[np.ndarray, np.ndarray]:
        bootstrap_indices = np.random.choice(len(self._X), size=len(self._X), replace=True)
        self._out_of_bag_X = self._X[np.setdiff1d(np.arange(len(self._X)), bootstrap_indices)]
        self._out_of_bag_y = self._y[np.setdiff1d(np.arange(len(self._X)), bootstrap_indices)]

        return self._X[bootstrap_indices], self._y[bootstrap_indices]

    @staticmethod
    def _select_crit(criterion: str) -> Callable:
        if criterion ==  "gini":
            return gini
        elif criterion == "entropy":
            return entropy
        else:
            raise ValueError("Unsupported criterion. Use 'gini' or 'entropy'.")

    def _validate_max_feat(self, max_features: Union[str, int]) -> int:
        if max_features == "auto":
            return int(np.sqrt(self._X.shape[1]))
        elif isinstance(max_features, int):
            return max_features
        else:
            raise ValueError("Unsupported max features count.")

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y

    def _split(self, points: np.ndarray, classes: np.ndarray, axis: int) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        left_mask = points[:, axis] == 0
        right_mask = ~left_mask
        return points[left_mask], points[right_mask], classes[left_mask], classes[right_mask]


    def _find_best_split(self, points: np.ndarray, classes: np.ndarray) -> int:
        best_dim = None
        best_gain = -np.inf

        feature_indices = np.random.choice(points.shape[1], size=self._max_features, replace=False)

        for dim in feature_indices:
            left_points, right_points, left_classes, right_classes = self._split(points, classes, dim)

            if len(left_classes) < self._min_samples_leaf or len(right_classes) < self._min_samples_leaf:
                continue

            current_gain = gain(left_classes, right_classes, self._criterion)
            if current_gain > best_gain:
                best_gain = current_gain
                best_dim = dim

        return best_dim

    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int) -> DecisionTreeNode:
        if (self._max_depth is not None and depth >= self._max_depth) or len(points) <= self._min_samples_leaf:
            return DecisionTreeLeaf(classes)

        split_dim = self._find_best_split(points, classes)
        if split_dim is None:
            return DecisionTreeLeaf(classes)

        left_points, right_points, left_classes, right_classes = self._split(points, classes, split_dim)

        return DecisionTreeInternalNode(
            split_dim,
            self._build_node(left_points, left_classes, depth + 1),
            self._build_node(right_points, right_classes, depth + 1))


    def _predict(self, points: np.ndarray, node: DecisionTreeNode) -> np.ndarray:
        if isinstance(node, DecisionTreeLeaf):
            return np.full(len(points), node.max_class)

        left_mask = points[:, node.split_dim] == 0
        right_mask = ~left_mask

        left_preds = self._predict(points[left_mask], node.left)
        right_preds = self._predict(points[right_mask], node.right)

        preds = np.empty(len(points), dtype=self._y.dtype)
        preds[left_mask] = left_preds
        preds[right_mask] = right_preds

        return preds

    def predict(self, points: np.ndarray) -> np.ndarray:
        return self._predict(points, self._root)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [ ]:
class RandomForestClassifier:

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self._criterion = criterion
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features
        self._n_estimators = n_estimators
        self._estimators = []

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features

    def fit(self, X, y) -> None:
        self._n_features = X.shape[1]
        self._estimators = [DecisionTree(X, y, criterion=self._criterion, max_depth=self._max_depth, min_samples_leaf=self._min_samples_leaf, max_features=self._max_features) for _ in range(self._n_estimators)]

    def predict(self, X) -> np.ndarray:
        predictions = np.array([tree.predict(X) for tree in self._estimators])
        return [mode(predicts) for predicts in predictions.T]

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [ ]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc):
    importances = np.zeros(rfc.n_features)

    for tree in rfc.estimators:
        X_oob, y_oob = tree.out_of_bag

        if len(X_oob) == 0:
            continue

        y_pred = tree.predict(X_oob)
        err_oob = 1 - accuracy_score(y_oob, y_pred)

        for j in range(rfc.n_features):
            X_permuted = X_oob.copy()
            np.random.shuffle(X_permuted[:, j])
            y_pred_permuted = tree.predict(X_permuted)
            err_oob_j = 1 - accuracy_score(y_oob, y_pred_permuted)

            importances[j] += err_oob_j - err_oob

        importances /= len(rfc.estimators)
        return importances


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [ ]:
def synthetic_dataset(size):
    X = [
        (np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, i % 6 == 0, i % 3 == 2, np.random.randint(0, 2))
        for i in range(size)
    ]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.45945946e-04
 3.70270270e-03 8.10810811e-05]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [ ]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [ ]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7175283732660782
Most important features:
1. ovsyanochan
2. styd.pozor
3. mudakoff
4. 4ch
5. nenormalnoo
6. rhymes
7. ohhluul
8. medieval_or
9. bog_memes
10. in.humour
11. kino_mania
12. ne1party
13. tumblr_vacuum
14. iwantyou
15. bot_maxim
16. club52205838
17. pixel_stickers
18. rapnewrap
19. fortnite
20. leprazo


#### Пол

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8524590163934426
Most important features:
1. 40kg
2. be.women
3. girlmeme
4. rapnewrap
5. reflexia_our_feelings
6. sh.cook
7. h.made
8. zerofat
9. 9o_6o_9o
10. cook_bon
11. 4ch
12. combovine
13. igm
14. soverwenstvo.decora
15. academyofman
16. i_d_t
17. be.beauty
18. i_want_love_dream
19. mudakoff
20. emptiness


### CatBoost
В качестве аьтернативы попробуем CatBoost.

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [ ]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(loss_function='MultiClass', verbose=0)
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

Accuracy: 1.0
Importance: [5.48456988e-03 1.16334730e-02 2.78558956e+01 2.78607784e+01
 4.42599666e+01 6.24136535e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [ ]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(
    X_train, y_age_train, y_sex_train, train_size=0.8
)

In [ ]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)


def get_best_params(y_train, y_eval):
    best_score, best_params = None, None
    for lr, md, msl in tqdm(list(product(learning_rate, max_depth, min_samples_leaf))):
        model = CatBoostClassifier(
            learning_rate=lr,
            depth=md,
            min_data_in_leaf=msl,
            loss_function='MultiClass',
            verbose=0
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_eval)
        score = accuracy_score(y_eval, y_pred)

        if best_score is None or score > best_score:
            best_score = score
            best_params = {"learning_rate": lr, "max_depth": md, "min_data_in_leaf": msl, "loss_function":'MultiClass', "verbose":0}

    return best_params, best_score

#### Возраст

In [ ]:
best_params, best_score = get_best_params(y_age_train, y_age_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075),
  'max_depth': 7,
  'min_data_in_leaf': 1,
  'loss_function': 'MultiClass',
  'verbose': 0},
 np.float64(0.7582340574632095))

In [ ]:
cb_model = CatBoostClassifier(**best_params)
cb_model.fit(X_train, y_age_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7477931904161412
Most important features:
1. mudakoff
2. dayvinchik
3. 4ch
4. rapnewrap
5. ovsyanochan
6. leprum
7. rhymes
8. fuck_humor
9. exclusive_muzic
10. kinomania


#### Пол

In [ ]:
best_params, best_score = get_best_params(y_sex_train, y_sex_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075),
  'max_depth': 4,
  'min_data_in_leaf': 1,
  'loss_function': 'MultiClass',
  'verbose': 0},
 np.float64(0.877365101611773))

In [ ]:
cb_model = CatBoostClassifier(**best_params)
cb_model.fit(X_train, y_sex_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8751576292559899
Most important features:
1. 40kg
2. mudakoff
3. modnailru
4. girlmeme
5. igm
6. femalemem
7. cook_good
8. be.beauty
9. 4ch
10. i_d_t
